# **Spatial typology to identify food and nutrition security bottlenecks in Niger**

**IFPRI- Calculate Productive Agriculture :Arable land computation**


```bash
Author : Aboubacar HEMA
Supervision : Wim MARIVOET
Contact : a.hema@cgiar.org / w.marivoet@cgiar.org
Role : Research Analyst at IFPRI
Year : December 2022
```

-----------------
## **Objective**
-----------------

- To identify arable land in country of interest at 30m resolution, comprising the following steps: 
1. Aggregate Cleared Forest and Cropland datasets 
2. Masking slope > 15%
3. permanent water
4. imprevious surfaces 


-----------------
## **Output**
-----------------

The estimation of total arable land by combining current crop land extent (Xiong et al., 2017) and total deforested areas (Hansen et al., 2013), of which the latter is assumed to be suitable for agriculture. To assure that arable land pixels do not fall within urbanized areas, permanent water bodies, or natural reserves, corresponding masks are used to obtain a final estimate of total arable land for each area (Brown de Colstoun et al., 2017; Pekel et al., 2016; UNEP-WCMC & IUCN, 2021).

-----------------
## **Dataset**
-----------------


 Directions for Downloading GFSAD Data:
1. Go to https://search.earthdata.nasa.gov/search. If you're a new user, create a login using contact information.
2. Search "GFSAD" in the top left search bar "Search for collections and topics"
3. Select "Global Food Security-support Analysis Data (GFSAD) Cropland Extent 2015 Africa 30 m V001"
4. In the map area to the right, image footprints will appear as on overlay over Africa. Navigate to country of interest, select image footprint that you would like to download. In the left side table showing search results, the image you have selected will be outlined in green (NOTE: This may require scrolling through the image list).
5. In the green outlined box, click "Download Granule" to download the scene to the desired output location on your computer.
6. Repeat these steps until you have downloaded all images required to cover your entire area of interest.
7. Upload download GFSAD geoTIFFs to GEE from Assets and import them into calcArableLand script.![image](https://userimages.githubusercontent.com/54441886/206823358-5993aad6-ae09-40d6-a33f-b14dd4f04bc7.png)

-----------------
## **Technical requirements**
-----------------


```bash
conda create -n gee python
conda activate gee
conda install -c conda-forge mamba
mamba install -c conda-forge pygis
pip install geemap
pip install ee
```


-----------------
## **Import the necessary libraries**
-----------------


In [44]:
import ee
import geemap

# Basic libraries of python for numeric and dataframe computations
import numpy as np                              
import pandas as pd

# Basic library for data visualization
import matplotlib.pyplot as plt 

# Slightly advanced library for data visualization            
import seaborn as sns 

In [45]:
# Authenticate and Initialize Earth Engine
geemap.ee_initialize()

-----------------
## **Load data**
-----------------

In [46]:
Hansen_GFC = ee.Image("UMD/hansen/global_forest_change_2020_v1_8")

surfaceWater = ee.Image("JRC/GSW1_4/GlobalSurfaceWater")

imperviousSurface = ee.Image("Tsinghua/FROM-GLC/GAIA/v10")

GFSAD30AFCE_2015_N_1 = ee.Image("projects/ee-aboubacarhema94/assets/Niger/GFSAD30AFCE_2015_N_1")
GFSAD30AFCE_2015_N_2 = ee.Image("projects/ee-aboubacarhema94/assets/Niger/GFSAD30AFCE_2015_N_2")
GFSAD30AFCE_2015_N_3 = ee.Image("projects/ee-aboubacarhema94/assets/Niger/GFSAD30AFCE_2015_N_3")
GFSAD30AFCE_2015_N_4 = ee.Image("projects/ee-aboubacarhema94/assets/Niger/GFSAD30AFCE_2015_N_4")

ImperviousSurfaceGMIS = ee.Image("projects/ee-aboubacarhema94/assets/Niger/NER_gmis_impervious_surface_percentage_geographic_30m")


ProtectedAreaPoints = ee.FeatureCollection("projects/ee-aboubacarhema94/assets/Niger/ProtectedAreaPoints")
ProtectedAreaPolygons = ee.FeatureCollection("projects/ee-aboubacarhema94/assets/Niger/ProtectedAreaPolygons")


GAUL_adm0 = ee.FeatureCollection("projects/ee-aboubacarhema94/assets/Niger/gadm41_NER_0")
NER_adm2 = ee.FeatureCollection("projects/ee-aboubacarhema94/assets/Niger/NER_64")

In [47]:
deforestCompYear = 14 #Last two digits of year only
occurrence_threshold = 10 #Set water occuring less than this value (in %) of the time, it is the frequency with which water was present.


In [48]:
adm0_AOI = GAUL_adm0
#Initialize a Map
Map = geemap.foliumap.Map()
Map.centerObject(adm0_AOI)
Map

In [49]:
# Set visualization/style parameters
adm2_style = {
    "fillColor": "00000000", # transparent color code
    "color": "black", # color of the stroke
    "width": 0.5 # stroke width
}

# Display the layer
Map.addLayer(NER_adm2.style(**adm2_style), {}, " Niger Administrative two Boundaries")
Map


-----------------
## **Global Food Security-support Analysis Data (GFSAD) Cropland Extent 2015 Africa 30 m V001**
-----------------

In [50]:
GFSAD_1 = GFSAD30AFCE_2015_N_1
GFSAD_2 = GFSAD30AFCE_2015_N_2
GFSAD_3 = GFSAD30AFCE_2015_N_3
GFSAD_4 = GFSAD30AFCE_2015_N_4

In [91]:
Img_stats = geemap.image_stats(GFSAD_1, scale=30)
Img_stats.getInfo()

{'max': {'b1': 2},
 'mean': {'b1': 1.187640515310589},
 'min': {'b1': 0},
 'std': {'b1': 0.39555668270727906},
 'sum': {'b1': 1647463005}}

In [92]:
Img_stats = geemap.image_stats(GFSAD_2, scale=30)
Img_stats.getInfo()

{'max': {'b1': 2},
 'mean': {'b1': 1.0820026743237259},
 'min': {'b1': 0},
 'std': {'b1': 0.28518047461927776},
 'sum': {'b1': 1500925031}}

In [ ]:
Img_stats = geemap.image_stats(GFSAD_3, scale=30)
Img_stats.getInfo()

In [ ]:
Img_stats = geemap.image_stats(GFSAD_4, scale=30)
Img_stats.getInfo()

In [51]:
viz = {
  'min': 0,
  'max': 2,
  'palette': [
    '014352', '1a492c', '071ec4',
  ]
}
Map.addLayer(GFSAD_1, viz, 'GFSAD 1', 1)

Map

In [52]:
Map.addLayer(GFSAD_2, viz, 'GFSAD 2', 1)

Map

In [53]:
Map.addLayer(GFSAD_3, viz, 'GFSAD 3', 1)
Map

In [11]:
Map.addLayer(GFSAD_4, viz, 'GFSAD 4', 1)
Map

Map(bottom=1979.0, center=[30.826780904779774, 8.393554687500002], controls=(WidgetControl(options=['position'…

In [12]:

#CLip Images to Niger Extent --> Mask
clipGFSAD_1 = GFSAD_1.clip(adm0_AOI)
clipGFSAD_2 = GFSAD_2.clip(adm0_AOI)
clipGFSAD_3 = GFSAD_3.clip(adm0_AOI)
clipGFSAD_4 = GFSAD_4.clip(adm0_AOI)
#Create Image Collection so it can be mosaiced together --> use quality mosaic to get highest
GFSAD = ee.ImageCollection([clipGFSAD_1, clipGFSAD_2, clipGFSAD_3, clipGFSAD_4]).mosaic()


In [101]:
'''
Map.addLayer(clipGFSAD_1, viz, 'GFSAD 1', 1)
Map.addLayer(clipGFSAD_2, viz, 'GFSAD 2', 1)
Map.addLayer(clipGFSAD_3, viz, 'GFSAD 3', 1)
Map.addLayer(clipGFSAD_4, viz, 'GFSAD 4', 1)
Map
'''

Map(bottom=3898.0, center=[21.37124437061832, 6.789550781250001], controls=(WidgetControl(options=['position',…

In [13]:
#Mask to select Cropland (band b1 = 2)
aoiCropland = GFSAD.eq(2).selfMask().rename("Cropland")

In [14]:
bin_viz = {
  'min': 0,
  'max': 1,
  'palette': ['red', 'green']
}

In [15]:
#Initialize a Map
Map = geemap.Map()
Map.centerObject(adm0_AOI)
Map.addLayer(aoiCropland, bin_viz, 'Cropland in GFSAD', 1)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [21]:
arableFromCropland = aoiCropland.select('Cropland').unmask(0).clip(adm0_AOI)


-----------------
## **Hansen Dataset for cleared forested between 2000 and 2015**
-----------------

In [22]:
#Initialize a Map
Map = geemap.Map()
Map.centerObject(adm0_AOI)

In [23]:
treeCoverVisParam = {
  'bands': ['treecover2000'],
  'min': 0,
  'max': 100,
  'palette': ['white', 'green']
}
Map.addLayer(Hansen_GFC, treeCoverVisParam, 'tree cover')
Map
treeLossVisParam = {
  'bands': ['lossyear'],
  'min': 0,
  'max': 20,
  'palette': ['yellow', 'red']
}
Map.addLayer(Hansen_GFC, treeLossVisParam, 'tree loss year')
Map

Map(center=[17.611001107213966, 8.08094566045597], controls=(WidgetControl(options=['position', 'transparent_b…

In [19]:
Hansen = Hansen_GFC.clipToCollection(adm0_AOI)
Map = geemap.Map()
Map.centerObject(adm0_AOI)
Map.addLayer(Hansen, treeLossVisParam, 'tree loss year')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [24]:
#Create Mask, selecting "loss year layer" less than or equal to 2015
aoiClearForestMask = Hansen.select("lossyear").lte(deforestCompYear)
#Mask Collection for only forests cleared between 2000 and 2015
aoiClearedForest = Hansen.mask(aoiClearForestMask)
aoiClearedForestLoss = aoiClearedForest.select('lossyear').gt(0).selfMask().rename("loss")
Map.addLayer(aoiClearedForestLoss, bin_viz, 'forest loss', 1)
Map


Map(bottom=7678.0, center=[17.611001107213966, 8.08094566045597], controls=(WidgetControl(options=['position',…

In [25]:
aoiClearedForestLoss = aoiClearedForestLoss.select('loss').unmask(0).clip(adm0_AOI)




-----------------
## **Global Man-made Impervious Surface (GMIS) Dataset**
-----------------

In [26]:
build_viz = {
  'bands': ['change_year_index'],
  'min': 0,
  'max': 34,
  'palette': [
    '014352', '1a492c', '071ec4', 'b5ca36', '729eac', '8ea5de',
    '818991', '62a3c3', 'ccf4fe', '74f0b9', '32bc55', 'c72144',
    '56613b', 'c14683', 'c31c25', '5f6253', '11bf85', 'a61b26',
    '99fbc5', '188aaa', 'c2d7f1', 'b7d9d8', '856f96', '109c6b',
    '2de3f4', '9a777d', '151796', 'c033d8', '510037', '640c21',
    '31a191', '223ab0', 'b692ac', '2de3f4',
  ]
}
Map = geemap.Map()
Map.centerObject(adm0_AOI)
Map.addLayer(imperviousSurface, build_viz, 'Change year index')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [27]:
build_viz = {
  'min': 0,
  'max': 34,
  'palette': [
    '014352',  'white',
  ]
}
Map = geemap.Map()
Map.centerObject(adm0_AOI)
Map.addLayer(ImperviousSurfaceGMIS, build_viz, 'impermeable Surface')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [76]:
geemap.image_stats(ImperviousSurfaceGMIS, scale=30).getInfo()

{'max': {'b1': 255},
 'mean': {'b1': 199.9730860292445},
 'min': {'b1': 0},
 'std': {'b1': 4.39442130607637},
 'sum': {'b1': 277514604874}}

In [28]:
GMIS_ImperviousSurface = ImperviousSurfaceGMIS
impermeableSurface = GMIS_ImperviousSurface.lte(99).unmask(0).clip(adm0_AOI)
#Initialize a Map
Map = geemap.Map()
Map.centerObject(adm0_AOI)
Map.addLayer(impermeableSurface, bin_viz, 'impermeable Surface', 1)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…


-----------------
## **Global Surface Water Dataset**
-----------------

In [29]:
visualization = {
  'bands': ['occurrence'],
  'min': 0.0,
  'max': 100.0,
  'palette': ['ffffff', 'ffbbbb', '0000ff']
}
#Initialize a Map
Map = geemap.Map()
Map.centerObject(adm0_AOI)
Map.addLayer(surfaceWater, visualization, 'Occurrence')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [30]:
#Clip JRC Image to Aoi and select "occurrence" band and unmask image so that nonwater pixels are 0
surfaceWater = surfaceWater.select('occurrence').clip(adm0_AOI)
#Initialize a Map
Map = geemap.Map()
Map.centerObject(adm0_AOI)
Map.addLayer(surfaceWater, visualization, 'Occurrence')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [31]:
WaterOccuring = surfaceWater.gt(occurrence_threshold).unmask(0).clip(adm0_AOI)
water_viz = {
  'min': 0,
  'max': 1,
  'palette': ['white', 'blue']
}
#Initialize a Map
Map = geemap.Map()
Map.centerObject(adm0_AOI)
Map.addLayer(WaterOccuring, water_viz, 'Water Occuring', 1)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…



-----------------
## **World’s protected areas**
-----------------

In [32]:
#Initialize a Map
Map = geemap.Map()
Map.centerObject(adm0_AOI)
Map.addLayer(ProtectedAreaPolygons, {'color': 'red'}, 'Protected Areas polygons',1)
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [33]:
Map.addLayer(ProtectedAreaPoints, {'color': 'red'}, 'Protected Areas points')
Map

Map(bottom=7678.0, center=[17.611001107213966, 8.08094566045597], controls=(WidgetControl(options=['position',…

In [34]:
inProtectedArea = 1
outProtectedArea = 0
outProtectedAreaImage = ee.Image(outProtectedArea).clip(adm0_AOI)
intProtectedAreaImage = ee.Image(inProtectedArea).clip(ProtectedAreaPolygons)
ProtectedAreaImage = outProtectedAreaImage.where(**{'test':intProtectedAreaImage, 'value':intProtectedAreaImage})
ProtectedAreaImage = ProtectedAreaImage.clip(adm0_AOI)

In [35]:
Map.addLayer(ProtectedAreaImage, bin_viz, 'Protected Area Image', 1)
Map

Map(bottom=7678.0, center=[17.611001107213966, 8.08094566045597], controls=(WidgetControl(options=['position',…

In [36]:
intProtectedAreaImage = ee.Image(inProtectedArea).clip(ProtectedAreaPoints)
PointsToImage = outProtectedAreaImage.where(**{'test':intProtectedAreaImage, 'value':intProtectedAreaImage})
PointsToImage = PointsToImage.clip(adm0_AOI)

In [37]:
Map.addLayer(PointsToImage, bin_viz, 'Points  Image', 1)
Map

Map(bottom=7678.0, center=[17.611001107213966, 8.08094566045597], controls=(WidgetControl(options=['position',…

In [38]:
aoiCropland = arableFromCropland.add(aoiClearedForestLoss).gte(1).selfMask().rename('arableLand')
arableLand_draft = aoiCropland.select('arableLand').unmask(0).clip(adm0_AOI)
arableLand_draft = arableLand_draft.subtract(ProtectedAreaImage).subtract(PointsToImage).subtract(WaterOccuring).subtract(impermeableSurface).rename('arableLand')
arableLand = arableLand_draft.eq(1).unmask(0).clip(adm0_AOI)


In [39]:
#Initialize a Map
Map = geemap.Map()
Map.centerObject(adm0_AOI)
Map.addLayer(arableLand, bin_viz, 'Arable land', 1)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [40]:
resolution = arableLand.projection().nominalScale()

In [41]:
def calculateFeatureSum(feature):
    areas = arableLand.reduceRegion(**{
    'reducer': ee.Reducer.sum(),
    'geometry': feature.geometry(),
    'scale': resolution,
   'maxPixels': 1e20
    })
    adm1_name = feature.get('NAME')
    return ee.Feature(
      feature.geometry(),
      areas.set('NAME', adm1_name))

In [42]:
#Map Function to Create
sumArable_byADM2 = NER_adm2.map(calculateFeatureSum)

In [43]:
df_sumArable_byADM2 = geemap.ee_to_df(sumArable_byADM2, sort_columns=True)

In [59]:
df_sumArable_byADM2.head()

,NAME,arableLand
0,Abala,1.930656e+05
1,Abalak,8.232675e+05
2,Aderbissanat,9.660157e+03
3,Agui�,2.247291e+06
4,Arlit,8.000000e+00


In [60]:
# Checking the info of the dataset
df_sumArable_byADM2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   NAME        64 non-null     object 
 1   arableLand  64 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.1+ KB


-----------------
## **Export to CSV**
-----------------

In [115]:
geemap.ee_to_csv(sumArable_byADM2, filename='Niger_Arable_land.csv') #Specify year

In [70]:
snippet = """
var visualization = {
  bands: ['change_year_index'],
  min: 0,
  max: 34,
  palette: [
    '014352', '1a492c', '071ec4', 'b5ca36', '729eac', '8ea5de',
    '818991', '62a3c3', 'ccf4fe', '74f0b9', '32bc55', 'c72144',
    '56613b', 'c14683', 'c31c25', '5f6253', '11bf85', 'a61b26',
    '99fbc5', '188aaa', 'c2d7f1', 'b7d9d8', '856f96', '109c6b',
    '2de3f4', '9a777d', '151796', 'c033d8', '510037', '640c21',
    '31a191', '223ab0', 'b692ac', '2de3f4',
  ]
};

Map.setCenter(-37.62, 25.8, 2);

Map.addLayer(dataset, visualization, 'Change year index');

"""

geemap.js_snippet_to_py(snippet, add_new_cell=True, import_ee=False)
